In [1]:
import numpy as np
import pandas as pd
import pulp as lp
import pickle as pkl
import time
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime as dt
import ALMPlanner as ALM
import ALMChart as ALMc
from ALMChart import standardized_chart

buyandhold_portfolios = ALM.load_scenario("scenario","eff_front")

In [2]:
######
EX = 1
######


if EX == 1:
    problem = ALM.ALMplanner(start = "2021", end = "2041", user_risk_profile = 0)#, buyandhold_portfolios="eff_front")
    problem.liabilities.insert("car", "2026", 25000, 25000*0.65)
    problem.liabilities.insert("university", "2029", 50000, 50000*0.95)
    problem.liabilities.insert("hawaii", "2037",25000, 25000*0.85) 
    problem.assets.insert("init","Jan 2021",30000)
    ALM.add_recurrent(problem, start = "Jan 2022", end = "Jan 2027", type = "asset", value = 10000, label = "ass")
elif EX == 2:
    problem = ALM.ALMplanner(start = "Jan 2021", end = "Jan 2061", user_risk_profile = 1)
    ALM.add_recurrent(problem, start = "Jan 2021", end = "Jan 2040", type = "asset", value = 1000, label = "ass")
    ALM.add_recurrent(problem, start = "Jan 2041", end = "Jan 2060", type = "goal", value_tg = 1500, value_lb = 1100, label = "ret")
elif EX == 3:
    problem = ALM.ALMplanner(start = "2021", end = "2041", user_risk_profile = 0)
    problem.liabilities.insert("car", "2036", 45000, 45000*0.65) 
    problem.assets.insert("init","Jan 2021",40000)
elif EX == 4:
    problem = ALM.ALMplanner(start = "2021", end = "2041", user_risk_profile = 0)#, buyandhold_portfolios="eff_front")
    problem.liabilities.insert("car", "2026", 30000, 30000*0.65)
    problem.liabilities.insert("university", "2029", 50000, 50000*0.95)
    problem.liabilities.insert("hawaii", "2037",30000, 30000*0.85) 
    problem.assets.insert("init","Jan 2021",30000)
    ALM.add_recurrent(problem, start = "Jan 2022", end = "Jan 2027", type = "asset", value = 10000, label = "ass")


In [3]:
ALMc.display(problem,bar_width = 6)

In [4]:
#problem.check_feasibility()
#problem.get_feasibility()

In [5]:
ALMc.display(problem,bar_width = 6)

In [6]:
GB_model = ALM.ALMGoalBased(problem)
GB_model.solve()

BaH_model = ALM.ALMBuyAndHold_2(problem)
BaH_model.solve()

GoalBased model generated in 8.84 s
Solve ended in 1.27 s with Optimal solution
Solve ended in 7.2 s with


# Display solution

In [7]:
sol = GB_model.solution
sol_bah = BaH_model.solution


In [8]:
P = problem.P
A = problem.assets.set
L = problem.liabilities.set
N = problem.N
T = problem.T
Scenario = problem.Scenario

In [9]:
colormap_liab = {}
colormap_ETF = {}
Ltot = list(L)
Ltot.append("extra_wealth")
it = -1
for e in Ltot:
    it = it+1
    colormap_liab[e] = px.colors.qualitative.Plotly[it%10]
it = -1

colormap_ETF = {P[0]:"royalblue", P[1]:"green", P[2]:"lime", P[3]:"yellow", P[4]:"orange", P[5]:"red"}

In [10]:
named_colorscales = px.colors.named_colorscales()
named_colorscales[0]

'aggrnyl'

In [11]:
plt = ALMc.AssetSplitDetailsChart(problem, sol, "Goal", colormap_liab)
plt.show()
plt = ALMc.AssetSplitDetailsChart(problem, sol_bah, "Goal", colormap_liab)
plt.show()

In [12]:
plt = ALMc.AssetSplitDetailsChart(problem, sol, "ETF", colormap_ETF)
plt.show()
plt = ALMc.AssetSplitDetailsChart(problem, sol_bah, "ETF", colormap_ETF)
plt.show()

In [13]:
n_scen = None
perc = False
#print([solution[l][n_scen].varValue for l in Lt])
plt = ALMc.AssetAllocationChart(problem,sol,n_scen=n_scen, perc = perc, portfolio_strategy = problem.user_portfolio)
plt.show()
plt = ALMc.AssetAllocationChart(problem,sol_bah,n_scen=n_scen, perc = perc, portfolio_strategy = problem.user_portfolio)
plt.show()

In [14]:
perc = False
prob, avg = ALMc.GoalRiskDetails(problem, sol, perc)
prob.show()
avg.show()

In [15]:
prob_bah, avg_bah = ALMc.GoalRiskDetails(problem, sol_bah, perc)
prob_bah.show()

avg_bah.show()

In [16]:
success_prob, fail_prob, tot_wealth_avg, paid_advance_avg, avg_loss_in_failure = ALMc.EoPWealthInfo(problem, sol)

print(f"\nPlan success/failure probability:")
print(f"   Success: {success_prob}%,\n   Failure: {fail_prob}%\n")

print(f"End-of-period wealth summary:")
print(f"                expected wealth: {int(tot_wealth_avg)}\n        thereof paid in advance: {int(paid_advance_avg)}\n   shortfall in case of failure: {int(avg_loss_in_failure)}")


Plan success/failure probability:
   Success: 94.8%,
   Failure: 5.2%

End-of-period wealth summary:
                expected wealth: 112824
        thereof paid in advance: 96303
   shortfall in case of failure: -872


In [17]:
success_prob, fail_prob, tot_wealth_avg, paid_advance_avg, avg_loss_in_failure = ALMc.EoPWealthInfo(problem, sol_bah)

print(f"\nPlan success/failure probability:")
print(f"   Success: {success_prob}%,\n   Failure: {fail_prob}%\n")

print(f"End-of-period wealt h summary:")
print(f"                expected wealth: {int(tot_wealth_avg)}\n        thereof paid in advance: {int(paid_advance_avg)}\n   shortfall in case of failure: {int(avg_loss_in_failure)}")


Plan success/failure probability:
   Success: 82.8%,
   Failure: 17.2%

End-of-period wealt h summary:
                expected wealth: 105405
        thereof paid in advance: 91639
   shortfall in case of failure: -1115


In [18]:
###############
###############
n_eff_front = 80
###############
###############

new_eff_front_portfolio = buyandhold_portfolios[n_eff_front]
sol.update_end(problem,buyandhold_portfolios[n_eff_front])
BaH_model2 = ALM.ALMBuyAndHold_2(problem)
BaH_model2.solve(buyandhold_portfolios[n_eff_front])
sol_bah = BaH_model2.solution

Solution updated in 4.09 s
Solve ended in 8.36 s with


In [26]:
fig = make_subplots(rows = 3, cols = 2, shared_yaxes= True)
n_scen = None
perc = False
plt = ALMc.AssetAllocationChart(problem,sol,n_scen=n_scen, perc = perc, portfolio_strategy = new_eff_front_portfolio)
fig.add_traces(data=plt.data, rows = 1, cols = 1)
plt = ALMc.AssetAllocationChart(problem,sol_bah,n_scen=n_scen, perc = perc, portfolio_strategy = new_eff_front_portfolio)
fig.add_traces(data=plt.data, rows = 1, cols = 2)
perc = False
prob, avg = ALMc.GoalRiskDetails(problem, sol, perc)
fig.add_traces(data=prob.data, rows = 2, cols = 1)
#fig.update_yaxes(range=[0, 1], row=2, col=1)
fig.add_traces(data=avg.data, rows = 3, cols = 1)
prob_bah, avg_bah = ALMc.GoalRiskDetails(problem, sol_bah, perc)
fig.add_traces(data=prob_bah.data, rows = 2, cols = 2)
fig.add_traces(data=avg_bah.data, rows = 3, cols = 2)
fig = ALMc.standardized_chart(fig)
fig.update_layout(barmode = "overlay")
fig

In [38]:
success_prob, fail_prob, tot_wealth_avg, paid_advance_avg, avg_loss_in_failure = ALMc.EoPWealthInfo(problem, sol)

print(f"\nPlan success/failure probability:")
print(f"   Success: {success_prob}%,\n   Failure: {fail_prob}%\n")

print(f"End-of-period wealth summary:")
print(f"                expected wealth: {int(tot_wealth_avg)}\n        thereof paid in advance: {int(paid_advance_avg)}\n   shortfall in case of failure: {int(avg_loss_in_failure)}")


Plan success/failure probability:
   Success: 94.8%,
   Failure: 5.2%

End-of-period wealth summary:
                expected wealth: 111622
        thereof paid in advance: 96303
   shortfall in case of failure: -872


In [39]:
success_prob, fail_prob, tot_wealth_avg, paid_advance_avg, avg_loss_in_failure = ALMc.EoPWealthInfo(problem, sol_bah)

print(f"\nPlan success/failure probability:")
print(f"   Success: {success_prob}%,\n   Failure: {fail_prob}%\n")

print(f"End-of-period wealth summary:")
print(f"                expected wealth: {int(tot_wealth_avg)}\n        thereof paid in advance: {int(paid_advance_avg)}\n   shortfall in case of failure: {int(avg_loss_in_failure)}")


Plan success/failure probability:
   Success: 80.5%,
   Failure: 19.5%

End-of-period wealth summary:
                expected wealth: 97646
        thereof paid in advance: 89977
   shortfall in case of failure: -666


In [ ]:
def show_named_plotly_colours():
    """
    function to display to user the colours to match plotly's named
    css colours.

    Reference:
        #https://community.plotly.com/t/plotly-colours-list/11730/3

    Returns:
        plotly dataframe with cell colour to match named colour name

    """
    s='''
        aliceblue, antiquewhite, aqua, aquamarine, azure,
        beige, bisque, black, blanchedalmond, blue,
        blueviolet, brown, burlywood, cadetblue,
        chartreuse, chocolate, coral, cornflowerblue,
        cornsilk, crimson, cyan, darkblue, darkcyan,
        darkgoldenrod, darkgray, darkgrey, darkgreen,
        darkkhaki, darkmagenta, darkolivegreen, darkorange,
        darkorchid, darkred, darksalmon, darkseagreen,
        darkslateblue, darkslategray, darkslategrey,
        darkturquoise, darkviolet, deeppink, deepskyblue,
        dimgray, dimgrey, dodgerblue, firebrick,
        floralwhite, forestgreen, fuchsia, gainsboro,
        ghostwhite, gold, goldenrod, gray, grey, green,
        greenyellow, honeydew, hotpink, indianred, indigo,
        ivory, khaki, lavender, lavenderblush, lawngreen,
        lemonchiffon, lightblue, lightcoral, lightcyan,
        lightgoldenrodyellow, lightgray, lightgrey,
        lightgreen, lightpink, lightsalmon, lightseagreen,
        lightskyblue, lightslategray, lightslategrey,
        lightsteelblue, lightyellow, lime, limegreen,
        linen, magenta, maroon, mediumaquamarine,
        mediumblue, mediumorchid, mediumpurple,
        mediumseagreen, mediumslateblue, mediumspringgreen,
        mediumturquoise, mediumvioletred, midnightblue,
        mintcream, mistyrose, moccasin, navajowhite, navy,
        oldlace, olive, olivedrab, orange, orangered,
        orchid, palegoldenrod, palegreen, paleturquoise,
        palevioletred, papayawhip, peachpuff, peru, pink,
        plum, powderblue, purple, red, rosybrown,
        royalblue, saddlebrown, salmon, sandybrown,
        seagreen, seashell, sienna, silver, skyblue,
        slateblue, slategray, slategrey, snow, springgreen,
        steelblue, tan, teal, thistle, tomato, turquoise,
        violet, wheat, white, whitesmoke, yellow,
        yellowgreen
        '''
    li=s.split(',')
    li=[l.replace('\n','') for l in li]
    li=[l.replace(' ','') for l in li]

    import pandas as pd
    import plotly.graph_objects as go

    df=pd.DataFrame.from_dict({'colour': li})
    fig = go.Figure(data=[go.Table(
      header=dict(
        values=["Plotly Named CSS colours"],
        line_color='black', fill_color='white',
        align='center', font=dict(color='black', size=14)
      ),
      cells=dict(
        values=[df.colour],
        line_color=[df.colour], fill_color=[df.colour],
        align='center', font=dict(color='black', size=11)
      ))
    ])

    fig.show()

show_named_plotly_colours()

In [ ]:
def AssetSplitDetailsChart(planner, solution, groupby, colormap):
    P = planner.P
    A = planner.assets.set
    L = planner.liabilities.set
    Assets_split = pd.DataFrame(index = np.arange(len(P)*len(A)*(len(L)+1)), columns = ["Asset", "Goal", "ETF", "Value"])
    it= -1
    for a in A:
        for p in P:
            for l in L:
                it = it+1
                Assets_split["Asset"][it] = a
                Assets_split["Goal"][it] = l
                Assets_split["ETF"][it] = p
                Assets_split["Value"][it] = solution.asset_to_goal[a][l][p]
            it = it+1
            Assets_split["Asset"][it] = a
            Assets_split["Goal"][it] = "extra_wealth"
            Assets_split["ETF"][it] = p
            Assets_split["Value"][it] = solution.asset_to_exwealth[a][p]
    AssetGroupedBy = Assets_split[["Asset", groupby, "Value"]].groupby(by=[groupby, "Asset"]).sum().reset_index()
    AssetPivot = AssetGroupedBy.pivot(index = "Asset", columns=groupby, values = "Value")
    AssetPivot["Period"] = pd.Series(planner.assets.period)
    AssetPivot = AssetPivot.sort_values(by = "Period")
    data = []
    if groupby == "Goal":
        groupby_set = list(L)
        groupby_set.append("extra_wealth")
    elif groupby == "ETF":
        groupby_set = P
    else:
        return
    data = []
    for e in groupby_set:
        data.append(go.Bar(x = AssetPivot.index, y = AssetPivot[e], marker_color = colormap[e], name = e))
    ASDChart = go.Figure(data = data, layout = go.Layout(barmode = "stack"))
    ASDChart = standardized_chart(ASDChart)
    return ASDChart